In [1]:
# No-Free-Miracle (Decimal-Digit Mapping) — v2 single-cell
# Reconciles your "identity audit" style with the conservative ledger:
#   - Sequences: Fibonacci word, Rabbit, Thue–Morse
#   - Mapping: 0.d1 d2 ... (decimal digits are {0,1})
#   - Pipelines: (sequence × (optional bitwise complement) × fixed transform)
#   - Selection at N_SELECT only; cross-N validation on N_HOLDOUTS (no reselection)
#   - Evidence ledger: Surplus ≈ 3.322 * D_train − log2(T)
#   - Nulls: random-sequence baseline across T; circular-shift baseline per winning pipeline

import math, random, statistics
from dataclasses import dataclass
from typing import List, Dict, Tuple, Callable, Optional

try:
    import mpmath as mp
except ImportError:
    import sys, subprocess; subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'mpmath'])
    import mpmath as mp

mp.mp.dps = 120  # working precision

# ------------------------------- Sequences (binary words) -------------------------------
def fib_word_bits(n: int) -> List[int]:
    s = "0"
    # Morphism: 0->01, 1->0
    while len(s) < n:
        s = s.replace('0','A').replace('1','B')
        s = s.replace('A','01').replace('B','0')
    s = s[:n]
    return [1 if c=='1' else 0 for c in s]

def rabbit_bits(n: int) -> List[int]:
    s = "1"
    # Morphism: 1->10, 0->1
    while len(s) < n:
        s = s.replace('0','A').replace('1','B')
        s = s.replace('A','1').replace('B','10')
    s = s[:n]
    return [1 if c=='1' else 0 for c in s]

def thue_morse_bits(n: int) -> List[int]:
    # Thue–Morse via parity-of-ones in binary index
    return [bin(k).count("1") & 1 for k in range(n)]

def complement_bits(bits: List[int]) -> List[int]:
    return [1-b for b in bits]

SEQ_FUNCS = {
    "FibonacciWord": fib_word_bits,
    "Rabbit":        rabbit_bits,
    "ThueMorse":     thue_morse_bits,
}

# ------------------------------ Map to decimal fraction ---------------------------------
def seq_to_decimal(bits: List[int]) -> mp.mpf:
    # Interpret 0.d1 d2 ... in base-10 where di ∈ {0,1}
    x = mp.mpf('0')
    w = mp.mpf('0.1')
    ten_inv = mp.mpf('0.1')
    for b in bits:
        if b == 1:
            x += w
        w *= ten_inv
    return x

# ------------------------------ Targets (fixed, predeclared) -----------------------------
def phi(): return (mp.mpf(1)+mp.sqrt(5))/2
ALPHA_INV_2018 = mp.mpf("137.035999084")
ALPHA_INV_2022 = mp.mpf("137.035999206")

targets: Dict[str, mp.mpf] = {
    "pi": mp.pi,
    "e": mp.e,
    "phi": phi(),
    "phi_minus_1": phi() - 1,
    "sqrt2": mp.sqrt(2),
    "sqrt3": mp.sqrt(3),
    "zeta3": mp.zeta(3),
    "CatalanG": mp.catalan,
    "Feigenbaum_delta": mp.mpf("4.66920160910299067185320382"),
    "alpha_inv_2018": ALPHA_INV_2018,
    "alpha_inv_2022": ALPHA_INV_2022,
    "alpha_2018": 1/ALPHA_INV_2018,
    "alpha_2022": 1/ALPHA_INV_2022,
}

# ------------------------------ Stage-2 transforms (fixed) -------------------------------
# Small, predeclared, mechanism-leaning set (keeps T honest).
def id_transform(x): return x
def reciprocal(x): return None if x == 0 else 1/x
def times_ten3_over_three6(x): return (mp.mpf(10)**3)/(mp.mpf(3)**6) * x    # 10^3 / 3^6
def times_sqrt3(x): return mp.sqrt(3) * x
def times_sqrt3_of_one_minus(x): return mp.sqrt(3) * (1 - x)                # for "B·√3" with B=1−F

STAGE2 = {
    "id": id_transform,
    "reciprocal": reciprocal,
    "times_10^3_over_3^6": times_ten3_over_three6,
    "times_sqrt3": times_sqrt3,
    "times_sqrt3_of_(1-x)": times_sqrt3_of_one_minus,
}

# ------------------------------- Pipelines (no N inside) ---------------------------------
@dataclass(frozen=True)
class Pipeline:
    seq_name: str
    complement: bool         # bitwise complement before decimal mapping
    transform_name: str

PIPELINES: List[Pipeline] = [
    Pipeline(sn, comp, tf)
    for sn in SEQ_FUNCS
    for comp in (False, True)
    for tf in STAGE2
]
T = len(PIPELINES)          # look-elsewhere count
LOG2_T = math.log2(T)

# ---------------------------- Scientific-digit comparison --------------------------------
def sci_digits(x: mp.mpf, M: int) -> Tuple[int, str]:
    """Return (exponent k, first M mantissa digits of |x|)."""
    if x == 0:
        return 0, "0"*M
    x = mp.fabs(x)
    k = int(mp.floor(mp.log10(x)))
    d = x / (mp.mpf(10) ** k)                  # 1 <= d < 10
    s = mp.nstr(d, n=M+1, strip_zeros=False)   # like '1.2345'
    if '.' in s:
        a, b = s.split('.'); digits = a + b
    else:
        digits = s
    if len(digits) < M: digits += "0"*(M-len(digits))
    return k, digits[:M]

def shared_prefix_len(a: str, b: str) -> int:
    m = min(len(a), len(b)); c = 0
    for i in range(m):
        if a[i] == b[i]: c += 1
        else: break
    return c

def digits_match(x: mp.mpf, y: mp.mpf, M: int, offset: int = 0) -> Tuple[int, int]:
    kx, sx = sci_digits(x, M+offset)
    ky, sy = sci_digits(y, M+offset)
    if kx != ky: return 0, 0
    return shared_prefix_len(sx[offset:], sy[offset:]), 1

# ------------------------------ Evaluation config ---------------------------------------
M_TOTAL = 50          # mantissa digits to compute
M_TRAIN = 14          # digits we allow for selection
M_TEST  = 14          # next held-out digits
assert M_TRAIN + M_TEST <= M_TOTAL

N_SELECT   = 1024     # N used for selecting pipelines
N_HOLDOUTS = [512, 2048]   # extra N for cross-N validation (no reselection)

# ------------------------------ Core evaluation ------------------------------------------
def run_pipeline(spec: Pipeline, N: int) -> Optional[mp.mpf]:
    bits = SEQ_FUNCS[spec.seq_name](N)
    if spec.complement:
        bits = complement_bits(bits)
    x = seq_to_decimal(bits)
    y = STAGE2[spec.transform_name](x)
    return y

def evaluate_selection():
    values: Dict[Pipeline, Optional[mp.mpf]] = {}
    for spec in PIPELINES:
        try:
            values[spec] = run_pipeline(spec, N_SELECT)
        except Exception:
            values[spec] = None

    results = {}
    for tname, tval in targets.items():
        best = None
        for spec, out in values.items():
            if out is None: continue
            D_train, exp_eq = digits_match(out, tval, M_TRAIN, offset=0)
            if exp_eq and (best is None or D_train > best["D_train"]):
                # compute held-out on the next M_TEST digits
                _, sx = sci_digits(out, M_TRAIN+M_TEST)
                _, sy = sci_digits(tval, M_TRAIN+M_TEST)
                kx, _ = sci_digits(out, 1); ky, _ = sci_digits(tval, 1)
                D_test = shared_prefix_len(sx[M_TRAIN:M_TRAIN+M_TEST], sy[M_TRAIN:M_TRAIN+M_TEST]) if kx==ky else 0
                best = {"spec": spec, "value": out, "D_train": D_train, "D_test": D_test}
        results[tname] = best
    return results

# ------------------------------ Nulls: random baselines ----------------------------------
def evaluate_random_baseline(num_runs: int = 30, seed: int = 123):
    rng = random.Random(seed)
    max_D_by_target = {t: [] for t in targets.keys()}
    for run in range(num_runs):
        values = {}
        for spec in PIPELINES:
            bits = [1 if rng.random() < 0.5 else 0 for _ in range(N_SELECT)]
            if spec.complement:
                bits = complement_bits(bits)
            x = seq_to_decimal(bits)
            y = STAGE2[spec.transform_name](x)
            values[spec] = y
        for tname, tval in targets.items():
            best_D = 0
            for out in values.values():
                if out is None: continue
                D, exp_eq = digits_match(out, tval, M_TRAIN, offset=0)
                if exp_eq and D > best_D: best_D = D
            max_D_by_target[tname].append(best_D)
    return {t: {
                "mean": statistics.mean(arr),
                "stdev": statistics.pstdev(arr),
                "max": max(arr),
                "hist": arr
            } for t, arr in max_D_by_target.items()}

# ------------------------------ Nulls: circular shifts -----------------------------------
def p_value_circular_shifts(spec: Pipeline, tval, num_samples: int = 200):
    base_bits = SEQ_FUNCS[spec.seq_name](N_SELECT)
    if spec.complement:
        base_bits = complement_bits(base_bits)
    # baseline D at N_SELECT
    base_val = STAGE2[spec.transform_name](seq_to_decimal(base_bits))
    base_D, exp_ok = digits_match(base_val, tval, M_TRAIN, offset=0)
    if not exp_ok: return float('nan'), base_D
    # sample shifts
    n = len(base_bits)
    step = max(1, (n-1)//num_samples)
    Ds = []
    for k in range(step, n, step):
        b = base_bits[k:] + base_bits[:k]
        val = STAGE2[spec.transform_name](seq_to_decimal(b))
        D, exp_ok2 = digits_match(val, tval, M_TRAIN, offset=0)
        if exp_ok2: Ds.append(D)
        if len(Ds) >= num_samples: break
    if not Ds: return float('nan'), base_D
    p = sum(1 for d in Ds if d >= base_D)/len(Ds)
    return p, base_D

# ------------------------------ Cross-N generalization -----------------------------------
def crossN_metrics(spec: Pipeline, tval, Ns: List[int]):
    Dtrains, Dtests = [], []
    exp_all = True
    for N in Ns:
        val = run_pipeline(spec, N)
        D, exp = digits_match(val, tval, M_TRAIN, offset=0)
        _, sx = sci_digits(val, M_TRAIN+M_TEST)
        _, sy = sci_digits(tval, M_TRAIN+M_TEST)
        kx, _ = sci_digits(val, 1); ky, _ = sci_digits(tval, 1)
        Dt = shared_prefix_len(sx[M_TRAIN:M_TRAIN+M_TEST], sy[M_TRAIN:M_TRAIN+M_TEST]) if kx==ky else 0
        Dtrains.append(D); Dtests.append(Dt)
        if exp == 0: exp_all = False
    return {
        "median_D_train": statistics.median(Dtrains),
        "min_D_train": min(Dtrains),
        "max_D_train": max(Dtrains),
        "median_D_test": statistics.median(Dtests),
        "exp_all_equal": exp_all,
    }

# ------------------------------ One-engine (anchor φ) ------------------------------------
def one_engine_anchor(anchor_target='phi'):
    results = evaluate_selection()
    info = results.get(anchor_target)
    if not info: return None, []
    spec = info["spec"]
    table = []
    for tname, tval in targets.items():
        val = run_pipeline(spec, N_SELECT)
        D, exp = digits_match(val, tval, M_TRAIN, offset=0)
        _, sx = sci_digits(val, M_TRAIN+M_TEST)
        _, sy = sci_digits(tval, M_TRAIN+M_TEST)
        kx, _ = sci_digits(val, 1); ky, _ = sci_digits(tval, 1)
        Dt = shared_prefix_len(sx[M_TRAIN:M_TRAIN+M_TEST], sy[M_TRAIN:M_TRAIN+M_TEST]) if kx==ky else 0
        table.append((tname, D, Dt))
    total_digits = sum(D for _, D, _ in table)
    return spec, table

# -------------------------------------- Run ----------------------------------------------
def main():
    print("=== No-Free-Miracle (decimal-digit mapping) ===")
    print(f"Pipelines T = {T}  → log2(T) = {LOG2_T:.2f} bits")
    print(f"Digits: TRAIN={M_TRAIN}, TEST={M_TEST} (out of {M_TOTAL}); N_SELECT={N_SELECT}; Holdouts={N_HOLDOUTS}\n")

    results = evaluate_selection()
    baseline = evaluate_random_baseline(num_runs=30, seed=42)

    rows = []
    for tname, info in results.items():
        if not info: continue
        spec = info["spec"]; Dtr = info["D_train"]; Dte = info["D_test"]
        surplus = 3.322*Dtr - LOG2_T
        # p vs random
        hist = baseline[tname]["hist"]; p_rand = sum(1 for d in hist if d >= Dtr)/len(hist)
        # p vs circular shifts
        p_shift, baseD = p_value_circular_shifts(spec, targets[tname], num_samples=200)
        # cross-N
        cross = crossN_metrics(spec, targets[tname], N_HOLDOUTS)
        rows.append({
            "target": tname,
            "pipeline": f"{spec.seq_name} | complement={'Y' if spec.complement else 'N'} | {spec.transform_name}",
            "D_train": Dtr, "D_test": Dte,
            "surplus_bits": surplus,
            "p_random": p_rand, "p_circshift": p_shift,
            "crossN_median_train": cross["median_D_train"],
            "crossN_median_test": cross["median_D_test"],
        })

    rows.sort(key=lambda r: (r["surplus_bits"], r["D_train"]), reverse=True)

    print("=== Top results by surplus bits (selection at N_SELECT) ===")
    for r in rows[:12]:
        print(f"- {r['target']:>18s}: D_train={r['D_train']:2d}, D_test={r['D_test']:2d}, "
              f"Surplus={r['surplus_bits']:6.2f} bits,  p_rand≈{r['p_random']:.3f}, p_shift≈{r['p_circshift'] if not math.isnan(r['p_circshift']) else float('nan'):.3f}")
        print(f"    pipeline: {r['pipeline']}")
        print(f"    cross-N  median D_train={r['crossN_median_train']}, median D_test={r['crossN_median_test']}\n")

    # One-engine → many-targets (anchor φ)
    spec_anchor, table = one_engine_anchor('phi')
    if spec_anchor:
        print("=== One-engine (chosen on φ) applied to all targets ===")
        print(f"engine: {spec_anchor.seq_name} | complement={'Y' if spec_anchor.complement else 'N'} | {spec_anchor.transform_name}")
        total = 0
        for t, D, Dt in table:
            total += D
            print(f"  {t:>18s}: D_train={D:2d}, D_test={Dt:2d}")
        print(f"  TOTAL digits across targets (train): {total}")

    print("\nNote: Surplus ≈ 3.322·D_train − log2(T). K=0 (no fitted continuous parameters). "
          "N is NOT part of T (we select at N_SELECT once, then validate on holdout Ns without reselection).")

if __name__ == "__main__":
    main()


=== No-Free-Miracle (decimal-digit mapping) ===
Pipelines T = 30  → log2(T) = 4.91 bits
Digits: TRAIN=14, TEST=14 (out of 50); N_SELECT=1024; Holdouts=[512, 2048]

=== Top results by surplus bits (selection at N_SELECT) ===
-              sqrt3: D_train= 2, D_test= 0, Surplus=  1.74 bits,  p_rand≈1.000, p_shift≈0.620
    pipeline: FibonacciWord | complement=N | times_sqrt3_of_(1-x)
    cross-N  median D_train=2.0, median D_test=0.0

-                phi: D_train= 1, D_test= 0, Surplus= -1.58 bits,  p_rand≈1.000, p_shift≈1.000
    pipeline: FibonacciWord | complement=N | times_sqrt3_of_(1-x)
    cross-N  median D_train=1.0, median D_test=0.0

-              sqrt2: D_train= 1, D_test= 0, Surplus= -1.58 bits,  p_rand≈1.000, p_shift≈1.000
    pipeline: FibonacciWord | complement=N | times_sqrt3_of_(1-x)
    cross-N  median D_train=1.0, median D_test=0.0

-              zeta3: D_train= 1, D_test= 0, Surplus= -1.58 bits,  p_rand≈1.000, p_shift≈1.000
    pipeline: FibonacciWord | complement=N